In [4]:
%load_ext sql
%sql sqlite:///PS2.db

u'Connected: None@PS2.db'

Problem Set 2
=======


### Instructions / Notes:

**_Read these carefully_**

* For some of the questions in this problem set _you will need to make your own instances of tables, either as solutions to the problems or for testing solutions to the problems_.
* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that your final answer for each question is _in its own cell_ and _clearly indicated_**
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging- i.e. running for too long: To restart the SQL connection, you must restart the entire python kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute the sql connection cell at top
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql [SQL]` is for _multi line_ SQL queries
* **See Piazza for submission instructions**
* _Have fun!_

Problem 1
---------

For Parts 1 & 2 of this problem you will need to provide a _single_ SQL query which will check whether a certain condition holds on the **hospital** table in the provided database:

In [5]:
%sql select * from hospital limit 3;

Done.


provider,hospital,address,city,state,zip,county,phone_number,hospital_type,hospital_owner,emergency_service,condition,measure_code
10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-CARD-2
10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-INF-1
10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-INF-2


You need to evaluate any requested conditions in the following way: **your query should return an empty result if and only if the condition holds on the instance.**  If the condition _doesn't hold_, your query should return something non-empty, but it doesn't matter what this is.

Note our language here: the conditions that we specify cannot be proved to hold **in general** without knowing the externally-defined functional dependencies; so what we mean is, _check whether they **are not violated** for the provided instance_.

You may assume that there are no `NULL` values in the tables.

### Part (a)

Is $\{provider\}$ a **superkey** for relation $Hospital$?

In [29]:
 %%sql
    SELECT *
    FROM hospital h1, hospital h2 
    WHERE h1.provider = h2.provider 
          AND (h1.hospital <> h2.hospital 
               OR h1.address <> h2.address 
               OR h1.city <> h2.city
               OR h1.state <> h2.state
               OR h1.zip <> h2.zip 
               OR h1.county <> h2.county 
               OR h1.phone_number <> h2.phone_number 
               OR h1.hospital_type <> h2.hospital_type 
               OR h1.hospital_owner <> h2.hospital_owner 
               OR h1.emergency_service <> h2.emergency_service
               OR h1.condition <> h2.condition 
               OR h1.measure_code <> h2.measure_code)
    LIMIT 1;
# NO

Done.


provider,hospital,address,city,state,zip,county,phone_number,hospital_type,hospital_owner,emergency_service,condition,measure_code,provider_1,hospital_1,address_1,city_1,state_1,zip_1,county_1,phone_number_1,hospital_type_1,hospital_owner_1,emergency_service_1,condition_1,measure_code_1
10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-CARD-2,10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Heart Attack,AMI-1


### Part (b)

Does $\{Zip\} \rightarrow \{City, State\}$ hold for relation $Hospital$?

In [43]:
%%sql 
SELECT *
FROM hospital h1, hospital h2
WHERE (h1.zip = h2.zip) AND (h1.city <> h2.city OR h1.state <> h2.state)
LIMIT 1;
# YES

Done.


provider,hospital,address,city,state,zip,county,phone_number,hospital_type,hospital_owner,emergency_service,condition,measure_code,provider_1,hospital_1,address_1,city_1,state_1,zip_1,county_1,phone_number_1,hospital_type_1,hospital_owner_1,emergency_service_1,condition_1,measure_code_1


### Part (c)

A **multivalued dependency (MVD)** is defined as follows: let $R$ be a schema i.e. a set of attributes, and consider two _sets of attributes_ $X\subseteq R$ and $Y\subseteq R$.  We say that a multivalued dependency (MVD), written:

$X\twoheadrightarrow Y$

**holds on $R$** if whenever there are two tuples $t_1,t_2$ such that $t_1[A] = t_2[A]$, there also exists a third tuple $t_3$ such that:

* $t_3[A] = t_1[A] = t_2[A]$
* $t_3[B] = t_1[B]$
* $t_3[R\setminus B] = t_2[R\setminus B]$

Note that $R\setminus B$ is all the attributes in $R$ that are not in $B$, and that $t_3$ need not be distinct from $t_1$ or $t_2$.  Note especially that an MVD holds on an entire _relation_, meaning that any two tuples (in any order) in the relation should satisfy the above conditions if the MVD holds.  **See the end of the lecture 8 slides for more on MVDs!**


For this part of this problem you will need to provide a _single_ SQL query which will check whether a MVD holds for relation $courses$.

In [3]:
%sql select * from courses limit 3;

Done.


course,book,lecturer
DB,Silberschatz,John D
DB,Nederpelt,John D
DB,Silberschatz,William M


Write your query to check if the MVD $\{course\}\twoheadrightarrow \{book\}$ holds for a relation $courses$.

In [47]:
%%sql
SELECT *
FROM courses c1, courses c2
WHERE c1.course = c2.course AND NOT EXISTS (
    SELECT *
    FROM courses c3
    WHERE c1.course = c3.course AND 
          c1.book = c3.book AND 
          c2.lecturer = c3.lecturer);

Done.


course,book,lecturer,course_1,book_1,lecturer_1


Problem 2
---------

Consider a relation $S(A,B,C,D,E,F)$ with the following functional dependencies:

* $\{A\} \rightarrow \{D\}$
* $\{A\} \rightarrow \{E\}$
* $\{D\} \rightarrow \{C\}$
* $\{D\} \rightarrow \{F\}$

In each part of this problem, we will examine different properties the provided schema (i.e., the provided relation with the above functional dependencies).

To answer **yes**, provide python code that assigns the variable ```answer``` to ```True``` and assigns ```explanation``` to be a python string which contains a (short!) explanation of why.  For example:

```python
answer = True
explanation = "Lise is correct because all keys are superkeys."
```

To answer **no**, provide python code that assigns the variable ```answer``` to ```False``` and assigns ```explanation``` to be a python string which contains a (short!) explanation of why.  For example:

```python
answer = False
explanation = "D is not a superkey because its closure is {D,C,F}."
```

### Part (a)

CS564 student Jeff claims that if ${A,B}$ is a superkey. Is Jeff correct?

In [ ]:
answer = True
explanation = "A+ = {A, C, D, E, F}, B+ = {B}, then {A, B}+ = {A, B, C, D, E, F}, which is the set of all attributes"

### Part (b)

Jeff also claims that the decomposition $ABC$, $CDE$, $EFA$ is lossless. Is Jeff correct?

In [ ]:
answer = False
explanation = "The intersection of CDE and EFA (which is E) is not candidtate key for any listed relations."

### Part (c)

CS564 Maria claims that the decomposition $ABC$, $CDE$, $EFA$ is dependency preserving. Is Maria correct?

In [ ]:
answer = False
explanation = "A → D not fit into the decompositions"

### Part (d)

Now consider a relation $R(X, Y, Z)$ with some list of functional dependencies $f_1, f_2, \ldots, f_n$. Now suppose that $K$ is a **key** for this relation, given these functional dependencies.

CS564 student Liam  claims that if we add any new functional dependency $f_{n+1} = U \rightarrow V$ to our list of functional dependencies, then $K$ will still be a key for $R$ given $f_1, f_2, \ldots, f_{n+1}$.  Is Liam correct? If yes, explain why.  If no, provide a counter-example in your explanation.

In [ ]:
answer = False
explanation = "If we have XY, {X, Y} -> {Z}, then Key is {X, Y}. When we add {X} -> {Y}, then only X is the Key.

Bonus Problem:
-------------

Prove the **transitivity rule for MVDs**

If $A\twoheadrightarrow B$ and $B\twoheadrightarrow C$ $\implies$ $A\twoheadrightarrow C \setminus B$

using only the basic definition of an MVD; and where $A,B,C$ are _sets of_ attributes such that $A\cup B\cup C\subseteq R$, where $R$ is the full set of attributes.